## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Saint-Philippe,RE,-21.3585,55.7679,67.59,69,50,15.75,scattered clouds
1,1,Yar-Sale,RU,66.8333,70.8333,44.53,69,99,15.03,overcast clouds
2,2,Hermanus,ZA,-34.4187,19.2345,54.81,80,40,18.59,scattered clouds
3,3,Lagoa,PT,39.0500,-27.9833,62.01,78,100,4.70,overcast clouds
4,4,Hasaki,JP,35.7333,140.8333,64.80,72,100,15.14,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
9,9,Vaini,TO,-21.2000,-175.2000,80.76,78,20,5.75,few clouds
13,13,Belmopan,BZ,17.2500,-88.7667,75.58,98,100,4.27,overcast clouds
17,17,Butaritari,KI,3.0707,172.7902,82.02,74,13,14.85,few clouds
27,27,Padang,ID,-0.9492,100.3543,84.25,76,100,0.11,overcast clouds
39,39,Kapaa,US,22.0752,-159.3190,84.18,79,100,17.27,overcast clouds
42,42,Shunyi,CN,40.1250,116.6453,76.95,57,40,6.71,scattered clouds
43,43,Veraval,IN,20.9000,70.3667,86.04,69,50,9.26,scattered clouds
46,46,Ternate,ID,0.8000,127.4000,85.55,63,89,2.68,overcast clouds
49,49,Hambantota,LK,6.1241,81.1185,86.34,70,20,9.22,few clouds
54,54,Palmerston,AU,-12.4860,130.9833,87.89,27,43,4.99,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                202
City                   202
Country                202
Lat                    202
Lng                    202
Max Temp               202
Humidity               202
Cloudiness             202
Wind Speed             202
Weather Description    202
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                202
City                   202
Country                202
Lat                    202
Lng                    202
Max Temp               202
Humidity               202
Cloudiness             202
Wind Speed             202
Weather Description    202
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
9,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,
13,Belmopan,BZ,75.58,overcast clouds,17.2500,-88.7667,
17,Butaritari,KI,82.02,few clouds,3.0707,172.7902,
27,Padang,ID,84.25,overcast clouds,-0.9492,100.3543,
39,Kapaa,US,84.18,overcast clouds,22.0752,-159.3190,
42,Shunyi,CN,76.95,scattered clouds,40.1250,116.6453,
43,Veraval,IN,86.04,scattered clouds,20.9000,70.3667,
46,Ternate,ID,85.55,overcast clouds,0.8000,127.4000,
49,Hambantota,LK,86.34,few clouds,6.1241,81.1185,
54,Palmerston,AU,87.89,scattered clouds,-12.4860,130.9833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):       
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
clean_hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
9,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort
13,Belmopan,BZ,75.58,overcast clouds,17.2500,-88.7667,Villa San Juan
17,Butaritari,KI,82.02,few clouds,3.0707,172.7902,Isles Sunset Lodge
27,Padang,ID,84.25,overcast clouds,-0.9492,100.3543,Mercure Padang
39,Kapaa,US,84.18,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
42,Shunyi,CN,76.95,scattered clouds,40.1250,116.6453,Beijing Mingchen Holiday Hotel
43,Veraval,IN,86.04,scattered clouds,20.9000,70.3667,Lords Inn Somnath
46,Ternate,ID,85.55,overcast clouds,0.8000,127.4000,Sahid Bela Ternate Hotel
49,Hambantota,LK,86.34,few clouds,6.1241,81.1185,Bungalow 63
54,Palmerston,AU,87.89,scattered clouds,-12.4860,130.9833,Quest Palmerston - Serviced Apartments and Acc...


In [23]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description} and {Max Temp} °F </dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))